In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
import os, os.path
import time
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as uReq
from urllib.request import Request
import requests
import regex as re
import unicodedata

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# folder path

In [3]:
# URL to scrap
url = 'https://fbref.com/en/matches/dadb1883/Spain-Costa-Rica-November-23-2022-World-Cup'

# headers
headers={'User-Agent': 'Mozilla/5.0'}

# requests
my_url = Request(url, headers=headers)
uClient = uReq(my_url)
time.sleep(0.5)
page_html = uClient.read()
time.sleep(0.5)
uClient.close()

# html parsing
soup = soup(page_html, "html.parser")

In [4]:
tables = soup.find_all('table')

In [5]:
# page_soup.findAll("div",{'id': 'team_stats'})
for table in soup.find_all('table'):
    print(table.get('class'))

None
None
None
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['min_width', 'sortable', 'stats_table', 'min_width', 'shade_zero']
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['stats_table', 'sortable']
['min_width', 'sortable', 'stats_table', 'min_width', 'shade_zero']
['stats_table', 'sortable', 'min_width']
['stats_table', 'sortable', 'min_width']


In [6]:
table = soup.find('table', class_='stats_table sortable')

In [7]:
df = pd.DataFrame(columns=['Player', '#', 'Pos', 'Age', 'Min',
                          'Gls', 'Ast', 'PK', 'PKatt', 'Sh', 'SoT', 'CrdY', 'CrdR', 'Touches', 'Tkl', 'Int', 'Blocks',
                          'xG', 'npxG', 'xAG',
                          'SCA','GCA', 'Cmp', 'Att', 'Cmp%', 'Prog',
                           'Succ', 'Att'])
df

,Player,#,Pos,Age,Min,Gls,Ast,PK,PKatt,Sh,...,npxG,xAG,SCA,GCA,Cmp,Att,Cmp%,Prog,Succ,Att


In [8]:
df2 = df[['Player', '#', 'Pos']]
df2

,Player,#,Pos


In [9]:
player_name = []
for name in table.tbody.find_all('th'):
    try:
        player_name.append(name.get_text(strip=True))
    except:
        print('error scraping tbody')
player_name = [unicodedata.normalize("NFKD", x) for x in player_name]    
counter = 0
for row in table.find_all('tr'):
    # Find data for each column
    columns = row.find_all('td')      
    # print(columns)        
    if columns != [] and counter <= 15:
        player = player_name[counter]        
        counter += 1        
        # print('counter', counter)
        number = columns[0].text.strip()        
        position = columns[1].text.strip()
        age = columns[2].text.split('-')[0]
        min_played = columns[3].text.strip()
        #print('age', type(age))
        # print(player, number, position)
        try:
            df2 = df2.append(
                {'Player': player,
                '#': number,
                'Pos': position,
                'Age': age,
                'Min': min_played
                }
                , ignore_index=True            
            )
            # print(f'Player {player} adicionado')
        except:
            print(f'error adding {player}')
df2

,Player,#,Pos,Age,Min
0,Marco Asensio,10,"FW,RW",26,68
1,Nico Williams,12,RW,20,22
2,Dani Olmo,21,LW,24,90
3,Ferrán Torres,11,RW,22,56
4,Álvaro Morata,7,FW,30,34
5,Pedri,26,LM,19,56
6,Carlos Soler,19,LM,25,34
7,Sergio Busquets,5,CM,34,63
8,Koke,8,CM,30,27
9,Gavi,9,RM,18,90
